# Desafio LATAM

Por Juan PINEDA-JARAMILLO
https://github.com/jdpinedaj


## Problema

El problema consiste en tomar el trabajo previo realizado por el Data Scientist y exponerlo para que sea explotado por un sistema


## Desarrollo


### 0. Importando las librerías y leyendo datos


In [4]:
import pandas as pd
import numpy as np

from datetime import datetime as dt
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as imbPipeline
from joblib import dump, load

# sklearn
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report
import xgboost as xgb
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.utils import resample

# settings to display all columns
pd.set_option("display.max_columns", None)

In [2]:
# Se lee el dataset original
df = pd.read_csv('../data/dataset_SCL.csv')


# Se generan las columnas adicionales, utilizando las siguientes funciones
def temporada_alta(fecha):
    """
    Función que calcula la temporada de alta de una fecha
    """
    fecha_año = int(fecha.split('-')[0])
    fecha = dt.strptime(fecha, '%Y-%m-%d %H:%M:%S')
    range1_min = dt.strptime('15-Dec', '%d-%b').replace(year=fecha_año)
    range1_max = dt.strptime('31-Dec', '%d-%b').replace(year=fecha_año)
    range2_min = dt.strptime('1-Jan', '%d-%b').replace(year=fecha_año)
    range2_max = dt.strptime('3-Mar', '%d-%b').replace(year=fecha_año)
    range3_min = dt.strptime('15-Jul', '%d-%b').replace(year=fecha_año)
    range3_max = dt.strptime('31-Jul', '%d-%b').replace(year=fecha_año)
    range4_min = dt.strptime('11-Sep', '%d-%b').replace(year=fecha_año)
    range4_max = dt.strptime('30-Sep', '%d-%b').replace(year=fecha_año)

    if ((fecha >= range1_min and fecha <= range1_max)
            or (fecha >= range2_min and fecha <= range2_max)
            or (fecha >= range3_min and fecha <= range3_max)
            or (fecha >= range4_min and fecha <= range4_max)):
        return 1
    else:
        return 0


def dif_min(data):
    """
    Función que calcula la diferencia de minutos entre una fecha y otra
    """
    fecha_o = dt.strptime(data['Fecha-O'], '%Y-%m-%d %H:%M:%S')
    fecha_i = dt.strptime(data['Fecha-I'], '%Y-%m-%d %H:%M:%S')
    dif_min = ((fecha_o - fecha_i).total_seconds()) / 60
    return dif_min


def get_periodo_dia(fecha):
    """
    Función que calcula el periodo de una fecha
    """
    fecha_time = dt.strptime(fecha, '%Y-%m-%d %H:%M:%S').time()
    mañana_min = dt.strptime("05:00", '%H:%M').time()
    mañana_max = dt.strptime("11:59", '%H:%M').time()
    tarde_min = dt.strptime("12:00", '%H:%M').time()
    tarde_max = dt.strptime("18:59", '%H:%M').time()
    noche_min1 = dt.strptime("19:00", '%H:%M').time()
    noche_max1 = dt.strptime("23:59", '%H:%M').time()
    noche_min2 = dt.strptime("00:00", '%H:%M').time()
    noche_max2 = dt.strptime("4:59", '%H:%M').time()

    if (fecha_time > mañana_min and fecha_time < mañana_max):
        return 'mañana'
    elif (fecha_time > tarde_min and fecha_time < tarde_max):
        return 'tarde'
    elif ((fecha_time > noche_min1 and fecha_time < noche_max1)
          or (fecha_time > noche_min2 and fecha_time < noche_max2)):
        return 'noche'


# Crerando las columnas adicionales, aplicando las funciones creadas anteriormente
df['temporada_alta'] = df['Fecha-I'].apply(temporada_alta)
df['dif_min'] = df.apply(dif_min, axis=1)
df['atraso_15'] = np.where(df['dif_min'] > 15, 1, 0)
df['periodo_dia'] = df['Fecha-I'].apply(get_periodo_dia)


/tmp/ipykernel_7663/1540584960.py:2: DtypeWarning: Columns (1,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../data/dataset_SCL.csv')


In [3]:
# Transformación de las variables realizadas por el data scientist

# Shuffle dataframe

data = shuffle(df[['OPERA', 'MES', 'TIPOVUELO', 'atraso_15']],
               random_state=111)

# Separar los datos de entrenamiento y prueba
features = data.drop(['atraso_15'], axis=1)
label = data['atraso_15']


### 1. Escoger el modelo que a tu criterio tenga un mejor performance, argumentando la decisión.


El data scientist entrenó un modelo de regresión logística y un XGBoost sencillos, verificando que su performance era muy malo.
Para mejorarlo, el data scientist optó por aplicar dos medidas:

- Encontrar los mejores hiperparámetros del modelo XGBoost mediante la aplicación de la metodología Grid Search.
- Balancear la clase minoritaria mediante la aplicación de la técnica de oversampling.

Esta elecci[on del data scientist es la mejor según el criterio de evaluación que ha elegido, por lo que se toma esta opción como inicial.


In [33]:
# Como el data scientist no imprimió los mejores parámetros que le daba el modelo,
# se realiza aquí nuevamente el grid search utilizando su misma técnica.

# Train test split
x_train, x_test, y_train, y_test = train_test_split(features,
                                                    label,
                                                    test_size=0.33,
                                                    random_state=42)

# Creación del Pipeline
## División de variables en numéricas y categóricas
categorical_features = ['OPERA', 'TIPOVUELO', 'MES']

## Definición de Transformadores
categorical_transformer = Pipeline(steps=[('encoder', OneHotEncoder())])

## Definición del preprocesador
preprocessor = ColumnTransformer(transformers=[('categorical',
                                                categorical_transformer,
                                                categorical_features)])

## Definición del Pipeline
pipeline = Pipeline(
    steps=[('preprocessor', preprocessor),
           ('classifier',
            xgb.XGBClassifier(random_state=1, learning_rate=0.01))])

# # Convirtiendo los features en un dataframe, conservando los nombres de las columnas como prefijos
# features = pd.DataFrame(features.toarray(),
#                         columns=enc.get_feature_names(
#                             ['OPERA', 'TIPOVUELO', 'MES']))

# Entrenamiento del modelo
modelxgb = pipeline.fit(x_train, y_train)


In [5]:
modelxgb

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['OPERA', 'TIPOVUELO',
                                                   'MES'])])),
                ('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, early_stopping_rounds=None,
                               enable_categorical=False, eval_met...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.01, max_bin=256,
                               max_cat_to_onehot=4, max_delta_step=0,
                               max_depth=6, max_leaves=0, min_child_weight=1,
                               missing=nan, monotone_constraints='()',
                               n_estimators=100, n_jobs=0, num_parallel_tree=1,
                               predictor='auto', random_state=1, reg_alpha=0,
                               reg_lambda=1, ...))])

In [6]:
# Definición de parámetros para implementar el grid search

parameters = {
    'classifier__learning_rate': [0.01, 0.05, 0.1],
    'classifier__n_estimators': [50, 100, 150],
    'classifier__subsample': [0.5, 0.9]
}

modelxgb_GridCV = GridSearchCV(modelxgb,
                               param_grid=parameters,
                               cv=2,
                               n_jobs=-1,
                               verbose=1).fit(x_train, y_train)

# Predicción del modelo
y_predxgb_grid = modelxgb_GridCV.predict(x_test)

# Confusion matrix
cfm = confusion_matrix(y_test.tolist(), y_predxgb_grid.tolist())
print(cfm)

# Imprimiendo los mejores parámetros
print(modelxgb_GridCV.best_params_)

Fitting 2 folds for each of 18 candidates, totalling 36 fits
[[18320    83]
 [ 3960   145]]
{'classifier__learning_rate': 0.05, 'classifier__n_estimators': 150, 'classifier__subsample': 0.9}


In [7]:
# Balanceo de datos usando misma metodología que el data scientist

data_no_retraso = data[data['atraso_15'] == 0]
data_atraso = data[data['atraso_15'] == 1]

# upsampling
data_atraso_upsampled = resample(
    data_atraso,
    replace=True,  # sample with replacement
    n_samples=30000,  # to match majority class
    random_state=42)  # reproducible results

data_upsampled = pd.concat([data_no_retraso, data_atraso_upsampled])

features_upsampled = data_upsampled.drop(['atraso_15'], axis=1)
label_upsampled = data_upsampled['atraso_15']


In [8]:
x_upsampled_train, x_upsampled_test, y_upsampled_train, y_upsampled_test = train_test_split(
    features_upsampled, label_upsampled, test_size=0.33, random_state=42)

# Entrenamiento del modelo
pipeline.fit(x_upsampled_train, y_upsampled_train)
y_upsampled_predxgb = modelxgb.predict(x_upsampled_test)
cfm_upsampled = confusion_matrix(y_upsampled_test.tolist(),
                                 y_upsampled_predxgb.tolist())
print(cfm_upsampled)


[[17315  1034]
 [ 8198  1699]]


In [9]:
print(classification_report(y_upsampled_test, y_upsampled_predxgb))

              precision    recall  f1-score   support

           0       0.68      0.94      0.79     18349
           1       0.62      0.17      0.27      9897

    accuracy                           0.67     28246
   macro avg       0.65      0.56      0.53     28246
weighted avg       0.66      0.67      0.61     28246



### 2. Implementar mejoras sobre el modelo escogiendo la o las técnicas que prefieras.


Las métricas obtenidas por el data scientist no son las mejores, considerando que lo que se busca predecir la probabilidad de que un vuelo esté atrasado, y este modelo tiene un recall del 0.19, lo cual es bastante malo. Así pues, es mejor tener un modelo que incrementé el recall para la clase que se busca predecir (en este caso la probabilidad de atraso de un vuelo, representado por el número: 1).

En un reto anterior de Data Scientist, utilizando datos adicionales de aeropuertos obtenidos en https://www.kaggle.com/datasets/jinbonnie/airport-information?resource=download, y creando variables adicionales como el conteo de número de vuelos atrasados por destino, número de vuelos atrasados por operador, número de vuelos atrasados por periodo del día, número de vuelos atrasados por periodo del día-mes-día, distancia entre el aeropuerto de Santiago de Chile y múltiples destinos (usando la fórmula de Haversine) logré entrenar un simple modelo de regresión logística con un recall de casi el 75%, muy superior al logrado por el data scientist.

Para más información, por favor mirar el notebook que creé en el reto anterior que se encuentra en mi repositorio: https://github.com/jdpinedaj/desafio_latam/blob/master/notebooks/solution.ipynb

Con el objetivo de no repetir todo el proceso que realicé ahí, que incluye incorporar nuevos datos y utilizar modelos distintos, tan solamente cambiaré algunos parámetros implementados allí para mejorar el modelo de XgBoost construido por el data scientist, siguiendo la filosofía de este desafío.


In [10]:
x_train

,OPERA,MES,TIPOVUELO
60899,Grupo LATAM,11,I
18403,Sky Airline,4,N
53808,Grupo LATAM,10,N
36508,Grupo LATAM,7,N
53445,Grupo LATAM,10,N
...,...,...,...
52990,Grupo LATAM,10,N
27973,Copa Air,6,I
60298,Grupo LATAM,11,I
10840,Grupo LATAM,2,I


In [11]:
# Modificar el pipeline para incluir SMOTE

sm = SMOTE()

# Creación del Pipeline
categorical_features = ['OPERA', 'TIPOVUELO', 'MES']
categorical_transformer = Pipeline(steps=[('encoder', OneHotEncoder())])
preprocessor = ColumnTransformer(transformers=[('categorical',
                                                categorical_transformer,
                                                categorical_features)])

## Definición del Pipeline, usando make_pipeline para imblearn como un paso
pipeline = imbPipeline(steps=[('preprocessor', preprocessor), (
    'smote',
    sm), ('classifier',
          xgb.XGBClassifier(random_state=1, learning_rate=0.01))])

# Entrenamiento del modelo
modelxgb_smote = pipeline.fit(x_train, y_train)

# Predicción del modelo
y_predxgb_smote = modelxgb_smote.predict(x_test)

print(classification_report(y_test, y_predxgb_smote))


              precision    recall  f1-score   support

           0       0.89      0.52      0.65     18403
           1       0.25      0.70      0.36      4105

    accuracy                           0.55     22508
   macro avg       0.57      0.61      0.51     22508
weighted avg       0.77      0.55      0.60     22508



In [12]:
modelxgb_smote.get_params

<bound method Pipeline.get_params of Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('categorical',
                                                  Pipeline(steps=[('encoder',
                                                                   OneHotEncoder())]),
                                                  ['OPERA', 'TIPOVUELO',
                                                   'MES'])])),
                ('smote', SMOTE()),
                ('classifier',
                 XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, early_stopping_rounds=None,
                               enable_categori...
                               gamma=0, gpu_id=-1, grow_policy='depthwise',
                               importance_type=None, interaction_constraints='',
                               learning_rate=0.01, max_bin

In [13]:
# Aplicación de la metodología Random Search para búsqueda de parámetros,
# puesto que esta metodología es más optima que el grid search.

# Definición de parámetros para implementar el random search

parameters = {
    'classifier__learning_rate': [0.01, 0.05, 0.1, 0.2, 0.3, 0.4, 0.5],
    'classifier__n_estimators': [50, 75, 100, 125, 150],
    'classifier__subsample': [0.5, 0.6, 0.7, 0.8],
    'classifier__max_depth': [3, 4, 5, 6, 7, 8, 9, 10],
    'classifier__min_child_weight': [1, 2, 3, 4, 5, 6],
    'classifier__gamma': [0.0, 0.1, 0.2, 0.3, 0.4],
    'classifier__colsample_bytree': [0.5, 0.6, 0.7]
}

modelxgb_smote_RandomCV = RandomizedSearchCV(
    modelxgb_smote,
    param_distributions=parameters,
    cv=5,  # Se incrementa el número de folds para que sea más robusto
    n_jobs=-1,
    n_iter=20,
    scoring=
    'recall',  # Se usa recall para que sea más robusto para la predicción de atrasos de vuelos
    verbose=1).fit(x_train, y_train)

# Predicción del modelo
y_predxgb_smote_rdm = modelxgb_smote_RandomCV.predict(x_test)

# Confusion matrix
cfm_rdm = confusion_matrix(y_test.tolist(), y_predxgb_smote_rdm.tolist())
print(cfm_rdm)

print(classification_report(y_test, y_predxgb_smote_rdm))

# Imprimiendo los mejores parámetros
print(modelxgb_smote_RandomCV.best_params_)


Fitting 5 folds for each of 20 candidates, totalling 100 fits
[[10367  8036]
 [ 1359  2746]]
              precision    recall  f1-score   support

           0       0.88      0.56      0.69     18403
           1       0.25      0.67      0.37      4105

    accuracy                           0.58     22508
   macro avg       0.57      0.62      0.53     22508
weighted avg       0.77      0.58      0.63     22508

{'classifier__subsample': 0.5, 'classifier__n_estimators': 100, 'classifier__min_child_weight': 6, 'classifier__max_depth': 8, 'classifier__learning_rate': 0.01, 'classifier__gamma': 0.2, 'classifier__colsample_bytree': 0.6}


Aquí se puede ver que las mejoras realizadas al data scientist mejoran las métricas (principalmente recall, que sería lo más importante según lo comentado previamente).
Sin embargo, y tal cual lo comenté anteriormente, en mi desafío anterior (https://github.com/jdpinedaj/desafio_latam/blob/master/notebooks/solution.ipynb) se consigue un mejor modelo mediante la utilización de datos adicionales.


In [17]:
# Aquí se vuelve a entrenar el nuevo y mejorado modelo xgboost al total de los datos,
# usando los mejores hiperparámetros encontrados mediante la aplicación de la metodología Random Search
#  y el oversampling con SMOTE

pipeline_final = imbPipeline(
    steps=[('preprocessor', preprocessor), ('smote', sm),
           ('classifier',
            xgb.XGBClassifier(random_state=1,
                              learning_rate=modelxgb_smote_RandomCV.
                              best_params_['classifier__learning_rate'],
                              n_estimators=modelxgb_smote_RandomCV.
                              best_params_['classifier__n_estimators'],
                              subsample=modelxgb_smote_RandomCV.
                              best_params_['classifier__subsample'],
                              max_depth=modelxgb_smote_RandomCV.
                              best_params_['classifier__max_depth'],
                              min_child_weight=modelxgb_smote_RandomCV.
                              best_params_['classifier__min_child_weight'],
                              gamma=modelxgb_smote_RandomCV.
                              best_params_['classifier__gamma'],
                              colsample_bytree=modelxgb_smote_RandomCV.
                              best_params_['classifier__colsample_bytree']))])

modelxgb_smote_RandomCV_final = pipeline_final.fit(features, label)

# Se guarda el modelo

dump(modelxgb_smote_RandomCV_final,
     "../models/modelxgb_smote_RandomCV_final.pkl")


['../models/modelxgb_smote_RandomCV_final.pkl']